In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
display(HTML("<style>.container { width:100% !important; }</style>"))

# FASE 0: Exploración y limpieza de los datos

**IMPORTACIÓN DE LIBRERIAS NECESARIAS PARA ESTA PRIMERA FASE**

In [3]:
import pandas as pd
import csv
import os
import numpy as np

**1. CARGA DE LOS DATOS**

A continuación se declara el dataframe cargado a través de un archivo csv llamado **200504COVID19MEXICO** en la ruta relativa "../data/200504COVID19MEXICO.csv", donde están contenidas variables tanto categóricas como numéricas. Se define el tipo de codificación **ISO-8859-1** para el alfabeto latino.


Este dataframe consta de 35 columnas y 100041 individuos. Cuatro variables numéricas referidas a fechas, **FECHA_ACTUALIZACIÓN**, **FECHA_INGRESO**, **FECHA_SINTOMAS** y **FECHA_DEF**, y las demás categricas.

In [4]:
data_covid = pd.read_csv("../../data/200504COVID19MEXICO.csv", encoding = "ISO-8859-1")

El siguiente dataframe con variables categóricas y numéricas se carga del csv llamado **CoordEstados** a través de la ruta relativa "../data/CoordEstados.csv".

Contiene un total de cinco variables, tres de ellas categóricas y dos numéricas, y un total de 36 entradas

In [5]:
data_estados = pd.read_csv("../../data/CoordEstados.csv", encoding = "ISO-8859-1")

El tercer dataframe se carga de un archivo en excel llamado **Catalogos_0412.xlsx** del que se carga la hoja llamada **Catálogo MUNICIPIOS**.

Consta de tres variables, todas ellas categóricas. Y 2501 entradas.

In [6]:
data_municipios = pd.read_excel('../../data/Catalogos_0412.xlsx','Catálogo MUNICIPIOS',
                                encoding = "ISO-8859-1")

Para poder tratar los datos en conjunto se realiza una unión de ambos dataframes por la variable **MUNICIPIO_RES** y **ENTIDAD_RES** que dará como resultado un dataframe llamado **df1**.

In [7]:
df1 = pd.merge(data_covid,data_municipios,left_on = ['MUNICIPIO_RES','ENTIDAD_RES'],
               right_on = ['CLAVE_MUNICIPIO','CLAVE_ENTIDAD'])

Se realiza la unión con el tercer dataframe obteniendo como resultado un único dataframe llamado **df**.

In [8]:
df = pd.merge(df1, data_estados[data_estados['Error'] == 'ok'], left_on = ['ENTIDAD_RES'], 
              right_on = ['Clave Estado'])

También se cuenta con un archivo json, llamado **mexico.json** con las coordenadas que limitan cada estado.

In [9]:
mexico_geo = r'../../data/mexico.json' 

**2. EXPLORACIÓN INICIAL DE LOS DATOS**

Dado que el dataset tiene un número asequible de columnas, antes de realizar una exploración más técnica es conveniente entender la información de la que se dispone. Por lo que se realiza una primera visualizacion del contenido completo que ofrece el dataset.

In [10]:
#Función de la librería para que muestre todas las columnas
pd.options.display.max_columns = None

In [11]:
df.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,INTUBADO,NEUMONIA,EDAD,NACIONALIDAD,EMBARAZO,HABLA_LENGUA_INDIG,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,CLAVE_MUNICIPIO,MUNICIPIO,CLAVE_ENTIDAD,Clave Estado,Estado,Latitud,Longitud,Error
0,04/05/2020,1ab4e5,2,3,25,2,25,25,6,2,06/04/2020,31/03/2020,10/04/2020,1,1,42,1,97,2,1,2,2,2,1,2,2,1,2,2,2,1,99,MÃ©xico,99,2,6,CULIACÁN,25,25,Sinaloa,25.0000001,-107.5000001,ok
1,04/05/2020,1448c7,2,3,25,2,99,25,6,2,14/04/2020,04/04/2020,17/04/2020,1,1,45,2,97,2,2,2,2,2,2,2,2,2,2,2,2,1,2,Estados Unidos de AmÃ©rica,99,2,6,CULIACÁN,25,25,Sinaloa,25.0000001,-107.5000001,ok
2,04/05/2020,73442,2,3,25,1,25,25,6,2,14/04/2020,06/04/2020,9999-99-99,2,1,50,1,2,2,2,2,2,2,2,2,2,1,2,2,2,1,99,MÃ©xico,99,2,6,CULIACÁN,25,25,Sinaloa,25.0000001,-107.5000001,ok
3,04/05/2020,07589e,2,3,25,2,9,25,6,1,23/04/2020,20/04/2020,9999-99-99,97,1,47,1,97,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,97,6,CULIACÁN,25,25,Sinaloa,25.0000001,-107.5000001,ok
4,04/05/2020,07eb77,2,3,25,2,25,25,6,2,27/04/2020,25/04/2020,9999-99-99,2,1,32,1,97,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,2,6,CULIACÁN,25,25,Sinaloa,25.0000001,-107.5000001,ok


In [12]:
#Función para enlistar todas las columnas y poder visualizarlas con mayor facilidad.
df.columns.to_list()

['FECHA_ACTUALIZACION',
 'ID_REGISTRO',
 'ORIGEN',
 'SECTOR',
 'ENTIDAD_UM',
 'SEXO',
 'ENTIDAD_NAC',
 'ENTIDAD_RES',
 'MUNICIPIO_RES',
 'TIPO_PACIENTE',
 'FECHA_INGRESO',
 'FECHA_SINTOMAS',
 'FECHA_DEF',
 'INTUBADO',
 'NEUMONIA',
 'EDAD',
 'NACIONALIDAD',
 'EMBARAZO',
 'HABLA_LENGUA_INDIG',
 'DIABETES',
 'EPOC',
 'ASMA',
 'INMUSUPR',
 'HIPERTENSION',
 'OTRA_COM',
 'CARDIOVASCULAR',
 'OBESIDAD',
 'RENAL_CRONICA',
 'TABAQUISMO',
 'OTRO_CASO',
 'RESULTADO',
 'MIGRANTE',
 'PAIS_NACIONALIDAD',
 'PAIS_ORIGEN',
 'UCI',
 'CLAVE_MUNICIPIO',
 'MUNICIPIO',
 'CLAVE_ENTIDAD',
 'Clave Estado',
 'Estado',
 'Latitud',
 'Longitud',
 'Error']

**VARIABLES SELECCIONADAS PARA ESTUDIO**

A continuación se declaran las variables consideradas importantes para la extracción de  conocimiento, su razonamiento y el contenido de cada una de ellas.

**Id_Registro**

Valor único de cada usuario. Se considera importante para el recuento de individuos.

---
**Sexo**

Identifica el sexo de cada paciente. Tiene importancia para la comprobación de diferencias entre sexos.

---
**Fecha_Ingreso**

Fecha de ingreso del paciente a la unidad de atención, por lo que se considera importante para cuantificar el transcurso del virus en el paciente.

---
**Fecha_Sintomas**

Revela la fecha en que se inició la sintomatología del paciente a la unidad de atención, por lo que se considera importante para cuantificar el transcurso del virus en el paciente.

---
**Fecha_Def**

Refleja la fecha en que el paciente falleció, se considera importante para cuantificar el transcurso del virus en el paciente.

---
**Intubado**

Describe si el paciente requirió de intubación. Con esta variable se obtienen datos de los requerimientos de cada paciente y la gravedad de la sintomatología.

---
**Neumonía**

Identifica si al paciente se le diagnosticó con neumonía debido al contagio por SARS CoV-2.

---
**Edad**

Contiene la información sobre la edad del paciente. Se considera un dato importante para analizar un perfil de riesgo si lo hubiera.

---
**Diabetes**

Refleja si el paciente esta diagnosticado de diabetes. Se le considera una variable de importancia para analizar su influencia en el contagio, en el fallecimiento y una comparación con las demás enfermedades.

---
**Epoc**

Contiene si el paciente está diagnosticado de enfermedad pulmonar obstructiva crónica. Se le considera una variable de importancia para analizar su influencia en el contagio, en el fallecimiento y una comparación a con las demás enfermedades.

---
**Asma**

Contiene si el paciente tiene un diagnóstico de asma. Se le considera una variable de importancia para analizar su influencia en el contagio, en el fallecimiento y una comparación con las demás enfermedades.

---
**Inmusupr**

Identifica si el paciente presenta inmunosupresión. Se le considera una variable de importancia para analizar su influencia en el contagio, en el fallecimiento y una comparación con las demás enfermedades.

---
**Hipertension**

Identifica si el paciente tiene un diagnóstico de hipertensión. Se le considera una variable de importancia para analizar su influencia en el contagio, en el fallecimiento y una comparación con las demás enfermedades.

---
**Otra_Com**

Refleja si el paciente tiene diagnóstico de otras enfermedades. Se le considera una variable de importancia para analizar su influencia en el contagio, en el fallecimiento y una comparación con las demás enfermedades.

---
**Cardiovascular**

Contiene si el paciente tiene un diagnóstico de asma. Se le considera una variable de importancia para analizar su influencia en el contagio, en el fallecimiento y una comparación con las demás enfermedades.

---
**Obesidad**

Contiene si el paciente tiene un diagnóstico de asma. Se le considera una variable de importancia para analizar su influencia en el contagio, en el fallecimiento y una comparación con las demás enfermedades.

---
**Renal_Cronica**

Contiene si el paciente tiene un diagnóstico de asma. Se le considera una variable de importancia para analizar su influencia en el contagio, en el fallecimiento y una comparación con las demás enfermedades.

---
**Tabaquismo**

Contiene si el paciente tiene un diagnóstico de asma. Se le considera una variable de importancia para analizar su influencia en el contagio, en el fallecimiento y una comparación con las demás enfermedades.

---
**Otro_caso**

Contiene información sobre si el paciente tuvo contacto con algún otro caso diagnósticado con SARS CoV-2. Por lo que sirve para analizar el porcentaje de rastreo.

---
**Resultado**

Identifica el resultado del análisis de la muestra reportado por el  laboratorio de la Red Nacional de Laboratorios de Vigilancia Epidemiológica (INDRE, LESP y LAVE).

---
**UCI**

Identifica si el paciente requirió ingresar a una Unidad de Cuidados Intensivos.

---
**Municipio**

Identifica el municipio de residencia del paciente.

---
**Entidad_res**

Identifica la entidad de residencia del paciente.

---
**Latitud**

Identifica la latitud de las corrdenadas cada estado de Méjico.

---
**Longitud**

Identifica la longitud de las corrdenadas de cada estado de Méjico

---


**Datos importantes que se pueden obtener del análisis**

Tras revisar los datos de los que se disponen, se considera importante analizar el comportamiento de la epidemia, la tasa de mortalidad, así como su comportamiento a lo largo del tiempo. Tambien se puede obtener de forma visual una discriminación por sexo, edad y enfermedades para obtener un perfil más claro de los pacientes con más riesgo. 

El objetivo del análisis que se quiere lograr es obtener una mayor comprensión el virus y un mayor conocimiento de su comportamiento 

Para ello se plantean las siguientes hipótesis:
- La edad influye en el fallecimiento de los diagnosticados con SARS CoV-2
- El sexo es un factor decisivo en el fallecimiento de los diagnosticados con SARS CoV-2
- Las enfermedades previas al contagio de SARS CoV-2 son un factor influye en el fallecimimento del paciente
- El Sexo influye en el diagnostico de positivos y negativos

**Comprobación de la existencia de valores duplicados:**

Para asegurar la fiabilidad del dataset, ya que no se han importado los datos directamente desde una consulta a la base de datos, sino que  han sido proporcionados por terceros y corren el riesgo de haber sido manipulados, se hará un control previo en base a la redundancia de información por registros duplicados. Para ello se comprobará la cantidad de valores totales que tenga cada uno de los registros por el identificador del registro.

In [13]:
x = print(df['ID_REGISTRO'].value_counts())

0.00E+00    35
1.86E+10     5
1.41E+03     4
1.94E+03     3
1.50E+86     3
            ..
1d1487       1
135062       1
087efe       1
19bc3d       1
13183        1
Name: ID_REGISTRO, Length: 99841, dtype: int64


Como se puede comprobar por los primeros elementos, existe tal riesgo ya que muestra 35 valores con **Id_Registro** valor 0 y algunos más de valor flotante. Para ello se comprueba si los valores corresponden a registros duplicados o se tratan de valores nulos.

In [14]:
df.sort_values(by='ID_REGISTRO')

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,INTUBADO,NEUMONIA,EDAD,NACIONALIDAD,EMBARAZO,HABLA_LENGUA_INDIG,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,CLAVE_MUNICIPIO,MUNICIPIO,CLAVE_ENTIDAD,Clave Estado,Estado,Latitud,Longitud,Error
86168,04/05/2020,0.00E+00,2,12,11,1,11,11,20,1,31/03/2020,27/03/2020,9999-99-99,97,2,48,1,2,2,2,2,2,2,2,2,2,2,2,2,1,2,99,MÃ©xico,99,97,20,LEÓN,11,11,Guanajuato,20.9876996,-101,ok
10839,04/05/2020,0.00E+00,2,12,9,2,9,15,104,1,28/04/2020,23/04/2020,9999-99-99,97,2,90,1,97,2,1,2,2,2,2,2,2,2,1,2,2,2,99,MÃ©xico,99,97,104,TLALNEPANTLA DE BAZ,15,15,Estado De México,19.4839446,-99.6899716,ok
41765,04/05/2020,0.00E+00,2,12,9,1,9,9,12,1,16/04/2020,16/04/2020,9999-99-99,97,2,38,1,2,2,2,2,2,2,2,2,2,2,2,2,1,2,99,MÃ©xico,99,97,12,TLALPAN,9,9,Ciudad De México,19.4326296,-99.1331785,ok
30399,04/05/2020,0.00E+00,2,4,14,2,14,14,97,2,10/04/2020,04/04/2020,13/04/2020,2,2,58,1,97,2,1,2,2,2,1,1,2,1,2,2,99,2,99,MÃ©xico,99,2,97,TLAJOMULCO DE ZÚÑIGA,14,14,Jalisco,20.3333331,-103.6666671,ok
53578,04/05/2020,0.00E+00,1,12,9,1,9,9,11,1,12/04/2020,10/04/2020,9999-99-99,97,2,58,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,99,MÃ©xico,99,97,11,TLÁHUAC,9,9,Ciudad De México,19.4326296,-99.1331785,ok
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77403,04/05/2020,99963,2,4,16,2,16,16,53,2,21/04/2020,21/04/2020,9999-99-99,2,2,42,1,97,2,2,2,2,2,2,2,2,2,2,2,99,2,99,MÃ©xico,99,2,53,MORELIA,16,16,Michoacán De Ocampo,19.207098,-101.878113,ok
7536,04/05/2020,99996,2,4,15,2,9,15,33,2,12/04/2020,08/04/2020,9999-99-99,2,1,50,1,97,2,2,2,2,2,2,2,2,1,2,2,99,1,99,MÃ©xico,99,2,33,ECATEPEC DE MORELOS,15,15,Estado De México,19.4839446,-99.6899716,ok
97446,04/05/2020,z166d5,1,12,1,1,1,1,1,2,17/04/2020,14/04/2020,9999-99-99,2,1,32,1,2,2,1,2,2,2,1,2,2,2,2,2,2,1,99,MÃ©xico,99,2,1,AGUASCALIENTES,1,1,Aguascalientes,21.8818,-102.291,ok
91116,04/05/2020,z21f6f,1,12,7,1,7,7,9,1,27/04/2020,24/04/2020,9999-99-99,97,99,40,1,2,2,2,1,2,2,2,2,2,2,2,2,2,1,99,MÃ©xico,99,97,9,ARRIAGA,7,7,Chiapas,16.5000001,-92.5000001,ok


In [15]:
len(df)

100039

In [16]:
len(df.drop_duplicates())

100039

Se concluye que son registros únicos pero existen valores nulos y con **Id_Registro** idénticos. Por lo que no es necesario tratarlos ya que no influye en el recuento de los individuos.

**Comprobación de la existencia de valores nulos**

Se llevará acabo una exploración de aquellos datos nulos que puedan darse en el dataset.

In [17]:
df.isnull().sum()

FECHA_ACTUALIZACION    0
ID_REGISTRO            0
ORIGEN                 0
SECTOR                 0
ENTIDAD_UM             0
SEXO                   0
ENTIDAD_NAC            0
ENTIDAD_RES            0
MUNICIPIO_RES          0
TIPO_PACIENTE          0
FECHA_INGRESO          0
FECHA_SINTOMAS         0
FECHA_DEF              0
INTUBADO               0
NEUMONIA               0
EDAD                   0
NACIONALIDAD           0
EMBARAZO               0
HABLA_LENGUA_INDIG     0
DIABETES               0
EPOC                   0
ASMA                   0
INMUSUPR               0
HIPERTENSION           0
OTRA_COM               0
CARDIOVASCULAR         0
OBESIDAD               0
RENAL_CRONICA          0
TABAQUISMO             0
OTRO_CASO              0
RESULTADO              0
MIGRANTE               0
PAIS_NACIONALIDAD      0
PAIS_ORIGEN            0
UCI                    0
CLAVE_MUNICIPIO        0
MUNICIPIO              0
CLAVE_ENTIDAD          0
Clave Estado           0
Estado                 0


**3. LIMPIEZA**

Llegados a este punto se puede coprobar que el dataset apenas requiere de limpieza pero sería conveniente elimiar al menos aquellas columnas que ya sabemos que pueden añadir ruido al futuro análisis. En este caso serán las columnas **Error** , **Clave_Municipio**, **Clave_Entidad** y **Clave Estado**.

In [18]:
df = df.drop(['Error', 'CLAVE_MUNICIPIO', 'CLAVE_ENTIDAD', 'Clave Estado'], axis=1)

Para unificar el formato de la información dada en el dataset se convierte a formato título los nombres de todas las columnas y los valores de la columna **Municipio**.

In [19]:
#Función para convertir una cadena en formato Título
df = df.rename(str.title, axis='columns')
df['Municipio'] = df['Municipio'].str.title()

Para el posterior uso del archivo **mexico.json** se normalizan los nombres de los municipios. Se eliminan las tildes de los registros en la columna **Municipios** del dataframe df. Y se cambia el nombre de **Ciudad de Mexico** por **CDMX**.

In [20]:
df['Municipio'] = df['Municipio'].str.normalize('NFKD').str.encode('ascii', errors='ignore')\
                .str.decode('utf-8')
df['Estado'] = df['Estado'].str.normalize('NFKD').str.encode('ascii', errors='ignore')\
                .str.decode('utf-8')
df = df.replace('Ciudad De Mexico','CDMX')

Para una mejor lectura de los resultados, se sustituyen los numeros de las categorias por nombres significativos.

In [21]:
df['Sexo'] = df['Sexo'].replace([1, 2, 99], ['Mujer', 'Hombre', 'No especificado'])

In [22]:
columnas = ['Intubado', 'Neumonia', 'Uci', 'Embarazo']
lista_enfermedades = list(df.columns)
lista = lista_enfermedades[19:29] + columnas

df[lista] = df[lista].replace([1, 2, 97, 98, 99], ['Si', 'No', 'No aplica', 'Se ignora', 'No especificado'])

In [23]:
df['Resultado'] = df['Resultado'].replace([1, 2, 3],['Positivo', 'Negativo',
                                                     'Resultado pendiente'])

In [24]:
df["Sector"]=df["Sector"].replace([np.nan,0,2,3,5,4,6,7,8,10,11,12,13],"Publico")
df["Sector"]=df["Sector"].replace([1,9],"Privado")
df["Sector"]=df["Sector"].replace(99,"No especificado")

**4. EXPORTACIÓN DE LOS DATOS**

Finalmente los datos serán exportados en un csv con el nombre de **dataset_covid2019** a la carpeta data que se encuentra en el repositorio del proyecto. Ruta relativa "../../data".

In [25]:
def exportación(df, name_file):
    path = '../../data'
    output = os.path.join(path, name_file)
    df.to_csv(output, index = False)

In [26]:
exportación(df, 'dataset_covid2019.csv')